<a href="https://colab.research.google.com/github/raegar/ImageProcessing/blob/main/ImageClassifierCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Preprocess the image function - you can modify this
def custom_preprocessing(image):
    # Default is to resize image to 64x64. You can expand this.
    processed_image = cv2.resize(image, (64, 64))
    return processed_image

# Load images and labels from a directory
def load_images_from_folder(folder, img_size=(64, 64)):
    images = []
    labels = []
    for class_label, class_folder in enumerate(os.listdir(folder)):
        class_folder_path = os.path.join(folder, class_folder)
        if os.path.isdir(class_folder_path):
            for filename in os.listdir(class_folder_path):
                img_path = os.path.join(class_folder_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = custom_preprocessing(img)  # Apply preprocessing
                    images.append(img)
                    labels.append(class_label)
    return np.array(images), np.array(labels)

# Create a simple CNN model
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),

        #HiddenLayers

        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  # Assuming 10 classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Plotting function to visualise accuracy and loss
def plot_results(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

# Function to display the first 10 images and show if they were classified correctly
def display_results(model, X_test, y_test, class_names):
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)

    # Plot the first 10 images
    plt.figure(figsize=(10, 10))
    for i in range(10):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X_test[i], cmap=plt.cm.binary)

        # Get the correct label and predicted label
        correct_label = class_names[y_test[i]]
        predicted_label = class_names[predicted_classes[i]]

        # If correctly classified, title will be in green; otherwise, in red
        color = 'green' if predicted_classes[i] == y_test[i] else 'red'
        plt.title(f"Pred: {predicted_label}\nTrue: {correct_label}", color=color)

    # Adjust subplot spacing to prevent text overlap
    plt.subplots_adjust(hspace=0.5)  # Increase the height between rows (adjust value as needed)

    plt.show()

# Main function
def main():
    # Load the dataset
    dataset_folder = '/content/drive/MyDrive/cats/train' #e.g. './data'
    images, labels = load_images_from_folder(dataset_folder)

    # Normalize pixel values
    images = images / 255.0

    # Define class names based on folder names (assuming folders are named by class)
    class_names = os.listdir(dataset_folder)

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Create the model
    input_shape = (64, 64, 3)
    model = create_model(input_shape)

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    # Plot the results
    plot_results(history)

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f'\nTest accuracy: {test_acc}')

    # Display the first 10 test images and whether they were correctly classified
    display_results(model, X_test, y_test, class_names)

if __name__ == '__main__':
    main()
